In [0]:
import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path="/content/drive/My Drive/Hackathon/SMSSpamCollection"

In [0]:
sms=pd.read_csv(path,sep='\t',names=['Label','Message'])

In [5]:
sms.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
sms.shape

(5572, 2)

In [7]:
sms['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [8]:
sms.isnull().values.any()

False

In [0]:
#sms.isnull().sum().sum()

0

In [0]:
nlp=spacy.load('en_core_web_sm')

In [0]:
#docsms=nlp(sms['Message'][0])

In [0]:
type(docsms)

spacy.tokens.doc.Doc

In [0]:
len(doc1)

20

# Clean function to remove punc and stopwords. Final step is to use lemmatized words

In [0]:
def clean_text(token):
  if token.is_stop==False and token.is_punct==False and not token.is_space and token.lemma_!='-PRON-':
    return True
  return False

def lemma_text(text):
    cleaned_text=[token.lemma_.lower() for token in nlp(text) if clean_text(token)]
    return cleaned_text


In [20]:
#sms['Message'].head(1).apply(lambda x: lemma_text(x))

0    [jurong, point, crazy, available, bugis, n, gr...
Name: Message, dtype: object

In [0]:
smscnt=CountVectorizer(analyzer=lemma_text,max_features=1000)
xcnt=smscnt.fit_transform(sms['Message'])


In [0]:
#test=nlp("$ + | data")
#for a in test:
 # if a.is_punct==False:
  #  print(a)

In [0]:
#smscnt.get_feature_names()

In [0]:
type(xcnt)

scipy.sparse.csr.csr_matrix

In [0]:
tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(xcnt)

In [24]:
X_train_tfidf.shape

(5572, 1000)

In [0]:
smsmb=MultinomialNB()

In [0]:
y=sms['Label']

In [0]:
x_train,x_test,y_train,y_test=train_test_split(X_train_tfidf,y,test_size=0.20,random_state=0)

In [29]:
x_train.shape

(4457, 1000)

In [30]:
type(x_train)

scipy.sparse.csr.csr_matrix

In [31]:
x_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.22745187,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [32]:
smsmb.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
y_predict=smsmb.predict(x_test)
print(y_predict)

['ham' 'spam' 'ham' ... 'ham' 'spam' 'ham']


In [34]:
np.mean(y_predict == y_test)

0.9838565022421525

In [35]:
from sklearn import metrics
metrics.confusion_matrix(y_test,y_predict)

array([[952,   3],
       [ 15, 145]])

In [36]:
print(metrics.classification_report(y_test,y_predict))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       955
        spam       0.98      0.91      0.94       160

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.97      1115
weighted avg       0.98      0.98      0.98      1115

